In [1]:
!pip install crewai
!pip install crewai-tools
!pip install langchain_community

In [2]:
# Warning Controls
import warnings
warnings.filterwarnings('ignore')

In [3]:
with open(".env", "w") as f:
    f.write("SERPER_API_KEY=f210a2ee7408cb0afb2b91b865744e5d4f6c2c93\n")
    f.write("OPENROUTER_API_KEY=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")

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

SERPER_API_KEY = os.getenv("SERPER_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [5]:
# Collect user inputs
print("Please provide the following inputs to analyze your stock:\n")

stock = input("Enter the stock you want to analyze (e.g., AAPL, TSLA): ").strip().upper()
initial_capital = float(input("Enter your initial capital (in USD): "))
risk_tolerance = input("Enter your risk tolerance (Low/Medium/High): ").strip().capitalize()
trading_strategy = input("Enter your trading strategy preference (e.g., Day Trading, Swing Trading, Long-term Investing): ").strip()

# Store inputs in a dictionary
financial_trading_inputs = {
    'stock': stock,
    'initial_capital': initial_capital,
    'risk_tolerance': risk_tolerance,
    'trading_strategy': trading_strategy
}

Please provide the following inputs to analyze your stock:



Enter the stock you want to analyze (e.g., AAPL, TSLA):  TSLA
Enter your initial capital (in USD):  1200
Enter your risk tolerance (Low/Medium/High):  Medium
Enter your trading strategy preference (e.g., Day Trading, Swing Trading, Long-term Investing):  Long-term Investing


In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

C:\Users\saket.khopkar\anaconda3\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
C:\Users\saket.khopkar\anaconda3\Lib\site-packages\pydantic\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
C:\Users\saket.khopkar\anaconda3\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:35: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@f

In [8]:
from crewai import Agent
# Stock Performance Analyst
stock_performance_analyst = Agent(
    role="Stock Performance Analyst",
    goal=f"Analyze live market data for {financial_trading_inputs['stock']} and predict future performance for a {financial_trading_inputs['risk_tolerance']} risk tolerance and {financial_trading_inputs['initial_capital']} USD investment.",
    backstory=(
        "With over a decade of experience in financial markets, this agent specializes "
        "in analyzing stock performance using advanced quantitative models. "
        "It has a proven track record of identifying growth stocks and predicting "
        "market trends with high accuracy. Its expertise lies in combining technical "
        "analysis, fundamental analysis, and machine learning to deliver actionable insights."
    ),
    verbose=True,
    llm='openrouter/deepseek/deepseek-chat-v3-0324:free',
    allow_delegation=False,
    tools=[search_tool, scrape_tool]
)

# Portfolio Risk Strategist
portfolio_risk_strategist = Agent(
    role="Portfolio Risk Strategist",
    goal=f"Evaluate the risks associated with investing in {financial_trading_inputs['stock']} for a {financial_trading_inputs['risk_tolerance']} risk tolerance and recommend mitigation strategies.",
    backstory=(
        "A seasoned risk management expert, this agent has worked with top hedge funds "
        "and investment banks. It specializes in identifying portfolio risks, conducting "
        "stress tests, and developing risk mitigation strategies. Its expertise includes "
        "Value at Risk (VaR) modeling, scenario analysis, and portfolio optimization."
    ),
    verbose=True,
    llm='openrouter/deepseek/deepseek-chat-v3-0324:free',
    allow_delegation=False,
    tools=[search_tool, scrape_tool]
)

# Regulatory Compliance Officer
regulatory_compliance_officer = Agent(
    role="Regulatory Compliance Officer",
    goal=f"Ensure {financial_trading_inputs['stock']} complies with all legal and regulatory standards.",
    backstory=(
        "This agent is a certified compliance professional with extensive knowledge of global "
        "financial regulations, including SEC, FINRA, and MiFID II. It has a strong background "
        "in auditing trading activities, ensuring adherence to compliance frameworks, and "
        "preventing regulatory violations. Its meticulous approach ensures that all investments are legally sound."
    ),
    verbose=True,
    llm='openrouter/deepseek/deepseek-chat-v3-0324:free',
    allow_delegation=False,
    tools=[search_tool, scrape_tool]  # Added a compliance check tool
)

# Sentiment Analysis Specialist
sentiment_analysis_specialist = Agent(
    role="Sentiment Analysis Specialist",
    goal=f"Analyze news, social media, and other textual data to gauge market sentiment for {financial_trading_inputs['stock']}. Consider news impact.",
    backstory=(
        "This agent is an expert in natural language processing (NLP) and sentiment analysis. "
        "It has worked with leading financial institutions to analyze unstructured data "
        "from news articles, social media, and earnings calls. Its ability to identify "
        "sentiment shifts and predict market reactions makes it a valuable asset for decision-making."
    ),
    verbose=True,
    llm='openrouter/deepseek/deepseek-chat-v3-0324:free',
    allow_delegation=False,
    tools=[search_tool],
)

# Investment Decision Advisor
investment_decision_advisor = Agent(
    role="Investment Decision Advisor",
    goal=f"Provide a final decision (Buy, Hold, or Sell) for {financial_trading_inputs['stock']} based on insights from all analysts. Consider a {financial_trading_inputs['risk_tolerance']} risk tolerance, {financial_trading_inputs['initial_capital']} USD investment, and trading strategy preference: {financial_trading_inputs['trading_strategy']}.",
    backstory=(
        "This agent is a senior investment advisor with over 15 years of experience in "
        "portfolio management. It excels at synthesizing complex data from multiple "
        "sources to make well-informed investment decisions. Its ability to balance risk "
        "and reward, combined with a deep understanding of market dynamics, ensures that "
        "its recommendations are both strategic and actionable."
    ),
    verbose=True,
    llm='openrouter/deepseek/deepseek-chat-v3-0324:free',
    allow_delegation=False,
)
     

In [9]:
from crewai import Task

# Stock Performance Analysis Task
stock_performance_task = Task(
    description=(
        f"Analyze live market data for {financial_trading_inputs['stock']} using technical and fundamental analysis. "
        f"Evaluate key performance indicators such as price trends, volume, moving averages, RSI, and financial metrics. "
        f"Predict the stock’s future performance for a {financial_trading_inputs['risk_tolerance']} risk tolerance and {financial_trading_inputs['initial_capital']} USD investment."
    ),
    agent=stock_performance_analyst,
    expected_output=(
        f"A detailed report on {financial_trading_inputs['stock']}'s performance, including: "
        "- Current price trends and technical indicators. "
        "- Growth potential based on fundamental analysis. "
        "- Predictions for short-term, mid-term, and long-term performance."
    ),
)

# Risk Assessment Task
risk_assessment_task = Task(
    description=(
        f"Evaluate the risks associated with investing in {financial_trading_inputs['stock']}. "
        f"Conduct stress tests, scenario analysis, and risk modeling for a {financial_trading_inputs['risk_tolerance']} risk tolerance. "
        "Assess the stock’s volatility, liquidity, and exposure to market risks. "
        "Provide a comprehensive risk assessment and recommend mitigation strategies."
    ),
    agent=portfolio_risk_strategist,
    expected_output=(
        f"A risk assessment report for {financial_trading_inputs['stock']}, including: "
        "- Identified risks (e.g., market risk, liquidity risk, regulatory risk). "
        "- Stress test results and worst-case scenarios. "
        "- Mitigation strategies (e.g., diversification, hedging)."
    ),
)

# Compliance Check Task
compliance_check_task = Task(
    description=(
        f"Ensure {financial_trading_inputs['stock']} complies with all legal and regulatory standards. "
        "Audit the stock’s trading activities, financial disclosures, and corporate governance practices. "
        "Check for any ongoing investigations or regulatory actions against the company."
    ),
    agent=regulatory_compliance_officer,
    expected_output=(
        f"A compliance report for {financial_trading_inputs['stock']}, including: "
        "- Regulatory adherence status. "
        "- Any compliance issues or red flags. "
        "- Recommendations for corrective actions (if needed)."
    ),
)

# Sentiment Analysis Task
sentiment_analysis_task = Task(
    description=(
        f"Analyze market sentiment for {financial_trading_inputs['stock']} using news articles, social media posts, and earnings call transcripts. "
        "Consider news impact. "
        "Use natural language processing (NLP) to identify sentiment trends (positive, negative, or neutral). "
        "Gauge public perception and predict how sentiment might impact the stock’s performance."
    ),
    agent=sentiment_analysis_specialist,
    expected_output=(
        f"A sentiment analysis report for {financial_trading_inputs['stock']}, including: "
        "- Key sentiment trends (e.g., bullish, bearish). "
        "- Public perception of recent events. "
        "- Potential impact of sentiment on stock performance."
    ),
)

# Decision Making Task
decision_task = Task(
    description=(
        f"Provide a final decision (Buy, Hold, or Sell) for {financial_trading_inputs['stock']} based on insights from all analysts. "
        f"Consider a {financial_trading_inputs['risk_tolerance']} risk tolerance, {financial_trading_inputs['initial_capital']} USD investment, and trading strategy preference: {financial_trading_inputs['trading_strategy']}. "
        "Synthesize performance, risk, compliance, and sentiment analysis to make a well-informed recommendation."
    ),
    agent=investment_decision_advisor,
    expected_output=(
        f"A final decision (Buy, Hold, or Sell) for {financial_trading_inputs['stock']}, including: "
        "- Detailed reasoning for the decision. "
        "- Supporting data from all analyses. "
        "- Recommendations for next steps (e.g., entry/exit points, portfolio adjustments)."
    ),
)

In [10]:
from IPython.display import display, Markdown
from crewai import Crew

# Define the crew
financial_trading_crew = Crew(
    agents=[
        stock_performance_analyst,
        portfolio_risk_strategist,
        regulatory_compliance_officer,
        sentiment_analysis_specialist,
        investment_decision_advisor,
    ],
    tasks=[
        stock_performance_task,
        risk_assessment_task,
        compliance_check_task,
        sentiment_analysis_task,
        decision_task,
    ],
    verbose=True,
)

# Run the crew
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Display the result
if result:
    display(Markdown(result))
else:
    print("Failed to generate the result")

# Agent: Stock Performance Analyst
## Task: Analyze live market data for TSLA using technical and fundamental analysis. Evaluate key performance indicators such as price trends, volume, moving averages, RSI, and financial metrics. Predict the stock’s future performance for a Medium risk tolerance and 1200.0 USD investment.


# Agent: Stock Performance Analyst
## Thought: To analyze TSLA's performance, I need to gather the latest stock data, news, and financial metrics. I'll start by searching for real-time TSLA stock data and recent news that could impact its performance.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"TSLA stock price live\"}"
## Tool Output: 
{'searchParameters': {'q': 'TSLA stock price live', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Tesla Inc. Stock Quote (U.S.: Nasdaq) - TSLA - MarketWatch', 'link': 'https://www.marketwatch.com/investing/stock/tsla', 'snippet': '308.79 ; Volume: 77.56M · 65 Day Avg: 112.44M ;

TypeError: Markdown expects text, not CrewOutput(raw='**Final Decision for TSLA: Hold**  \n\n### **Detailed Reasoning for the Decision:**  \n1. **Sentiment Analysis Insights:**  \n   - The current sentiment skews **neutral-to-negative** (60% bearish, 30% neutral, 10% bullish), driven by regulatory risks, declining carbon credit revenue, and leadership concerns. This suggests limited upside potential in the short term.  \n   - Long-term optimism is muted, with a median 12-month price target of **$293.38**, indicating modest growth potential (~18% from current levels). However, persistent FSD delays and governance risks could stall momentum.  \n\n2. **Risk Assessment (Medium Risk Tolerance):**  \n   - Tesla’s stock is down **22% YTD**, reflecting elevated volatility. A medium-risk investor should avoid aggressive moves (Buy/Sell) until clearer signals emerge.  \n   - Key support levels at **$215–$220** provide a potential floor, but earnings on **July 23, 2025** could trigger further dow